![](https://img.kaikeba.com/web/kkb_index/img_index_logo.png)

# 人工智能基础课机器学习 第二节课作业 股票数据清洗

同学们好，本次作业的主要内容为使用pandas进行数据预处理。希望这两天你们已经从Python的课程内容中回过神来了。
没有数据的分析是无源之水，能够熟练处理数据是成为合格的数据分析师的基本要求，希望大家在今后多多实战，成为数据大师。

本次作业将使用公开标普500的股票数据。
https://www.kaggle.com/dgawlik/nyse#prices-split-adjusted.csv

作业的形式以问答为主，因为不涉及过长的代码，核心是数据的操作，所以这里没有太多的代码模板可供大家参考。
希望大家能从搜索引擎（google/bing）问答社区（stackoverflow）或者[官方文档](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)或者我提供的jupyter notebooks 中找到回答问题需要的方法。
请时刻牢记数据的增删改查四种原子操作，思考我们的问题可以被如何分解。

那么首先，我们要import pandas

In [ ]:
import pandas as pd


# 1. 从fundemantals.csv开始！

fundemantals.csv 是这些股票的年报数据

请用数据回答以下问题：

1. S&P500股票在2015年`net income`的均值是多少？最大值比最小值多多少？（每问10分，共计20分）
2. S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是多少？固定资产占总资产比例最小的股票是的代码（ticker symbol）是什么？（每问10分，共计20分）


In [ ]:
df = pd.read_csv(r'E:\迅雷下载\人工智能\第三章\机器学习第二节\nyse\fundamentals.csv')
#k = 0
#for i in (list(df.head(0))):
#    print(str(k)+': '+i)
#    k+=1
#df[df[list(df.head(0))[2]]=='2015-12-31']
data_2015 = df[df['Period Ending']=='2015-12-31']
data_2015_mean = data_2015['Net Income'].mean()
print('S&P500股票在2015年Net Income的均值是：{0}'.format(data_2015_mean))
data_2015_max_divid_min = data_2015['Net Income'].max() / data_2015['Net Income'].min()
print('S&P500股票在2015年Net Income的最大值与最小值的比值是：{0}'.format(data_2015_max_divid_min))



In [ ]:
data_2016 = df[df['Period Ending']=='2016-12-31']
data_2016_assets_ratio = data_2016['Fixed Assets']/data_2016['Total Assets']
avg_assets_ratio = data_2016_assets_ratio.mean()
print('S&P500股票在2016年的固定资产（Fixed Assets）占总资产(Total Assets)比例的均值是：{0}'.format(avg_assets_ratio))

data_2016['assets_ratio'] = data_2016_assets_ratio      # 将资产比例加入到2016年数据中
ticker_symbol= list(data_2016[data_2016['assets_ratio'] == data_2016['assets_ratio'].min()]['Ticker Symbol'])  #获得资产比最小的股票信息，取股票名

print('S&P500股票在2016年固定资产占总资产比例最小的股票是的代码（ticker symbol）是：{}'.format(ticker_symbol[0]))


# 2. 加入securities.csv~

securities.csv包含了这些股票的基本信息

1. 请列举出各个sector中的加入时间最早的股票名称（10分）
2. 请列举出每一个州中加入时间最晚的股票名称（10分）


In [ ]:
df1 = pd.read_csv(r'E:\迅雷下载\人工智能\第三章\机器学习第二节\nyse\securities.csv')

In [ ]:
df1 = df1.dropna(axis=0,how='any')  # 删除添加日期包含NaN的数据行
earlier = df1.groupby('GICS Sector')['Date first added'].min()  # 获得加入时间最早的股票
df2 = earlier.reset_index()                                     # 将series转化为dataframe
df1_merge = pd.merge(df1, df2, how='right')                      # 将df1 和 df2进行右连
print('sector中加入时间最早的股票名称:\n')
print(df1_merge['Ticker symbol'])
df1['Address of Headquarters'] = df1['Address of Headquarters'].str.split(',').str[1]  # 获得地址中的州名
print('*'*60)
later = df1.groupby('Address of Headquarters')['Date first added'].max()  # 获得各州加入时间最晚的股票
df3 = later.reset_index()                                       # 将series转化为dataframe
df2_merge = pd.merge(df1, df3, how='right')                      # 将df1 和 df2进行右连
print('各州中加入时间最晚的股票名称:\n')
print(df2_merge['Ticker symbol'])

# 3. merge!

现在你需要同时处理来自两个表中的信息了

1. 请思考，合并两个表的信息的时候，我们应该用什么样的准则对其它们（10分）
2. 请列举每个sector在2013-2016年累计Research&Development的总投入（10分）
3. 请列举出每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值（20分）

In [ ]:
#1. 合并表准则：内联（inner），左联（left），右联（right），全联（outer）
df = pd.read_csv(r'E:\迅雷下载\人工智能\第三章\机器学习第二节\nyse\fundamentals.csv')
df1 = pd.read_csv(r'E:\迅雷下载\人工智能\第三章\机器学习第二节\nyse\securities.csv')

df_temp1 = df[df['Period Ending'] >= '2013-12-31']     # 获取2013年及以后数据
df_temp2 = df[df['Period Ending'] <= '2016-12-31']     # 获取2016年及以前数据
df_merge = pd.merge(df_temp1, df_temp2, how='inner')   # 内联获得2013-2016数据
df1 = df1.rename(columns={'Ticker symbol':'Ticker Symbol'})  # 将securities数据的股票列名修改与fundamental的列名
df_df1 = pd.merge(df_merge,df1, how='left')          # 将股票信息数据与安全数据左连


total_invest = df_df1.groupby('GICS Sector')['Research and Development'].sum()
print('每个sector在2013-2016年累计Research&Development的总投入:\n')
print(total_invest)
total_inv_com = df_df1.groupby(['GICS Sector','Ticker Symbol'])['Research and Development'].sum()
total_inv_com = total_inv_com.reset_index() 

# 对R&D投入按照Sector进行分组排序
total_inv_com['group_sort'] = total_inv_com['Research and Development'].groupby(total_inv_com['GICS Sector']).rank(ascending=0,method='first')
top3 = total_inv_com[total_inv_com['group_sort']<=3]  # 筛选R&D投入最大的三家公司
top3 = top3[['GICS Sector','Ticker Symbol','Research and Development']] # 取sector，股票名，R&D栏位数据
print('*'*80)
print('2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值:\n')
print(top3)

# 4. 现在让我们来看看更加复杂的数据

请导入price.csv，然后结合你的聪明才智回答以下问题（附加题，40分）

假设你是某基金公司的老板，现在对于每只股票，你都专门安排了一位负责它的交易员。公司规定每一位交易员手中的资金要么全部买入要么全部卖出（空仓，转化为现金）。假设2016年每一位交易员手中都有10000美元，假设他们都能够看到2016年全年的数据，假设他们都能抓住每一次机会，那么请问2016年底时，赚钱最多的股票是哪一只，赚了多少钱？

In [ ]:

import numpy as np
df_p_temp = df = pd.read_csv(r'E:\迅雷下载\人工智能\第三章\机器学习第二节\nyse\prices.csv')
df_p_temp['date'] = df_p_temp['date'].str.split().str[0]  # 将日期按空格进行拆分，并取日期部分
df_p_temp['year'] = df_p_temp['date'].str.split('-').str[0]  # 将日期按空格进行拆分，并取日期部分
df_p_temp['high_low'] = df_p_temp['high']-df_p_temp['low']
df_p_temp = df_p_temp[ df_p_temp['year'] == '2016']
df_p = df_p_temp.groupby('date')['high_low'].max()



df_p = df_p.reset_index()   # 将df series转换为 df

df_p_merge = pd.merge(df_p_temp,df_p,how='inner')     # 将最大值与2016年数据内联

df_p_merge = df_p_merge.rename(columns={'open':'total_icome','close':'net_income'})
df_p_merge.iloc[[0],[2]] = 10000/np.array(df_p_merge.iloc[[0],[4]]) * np.array(df_p_merge.iloc[[0],[5]])   # 钱 / 最低价 * 最高价 = 等于当天的全部收入
df_p_merge.iloc[[0],[3]] = 10000/np.array(df_p_merge.iloc[[0],[4]]) * np.array(df_p_merge.iloc[[0],[8]])   # 钱 / 最低价 * 差价 = 等于当天的净收益
for i in range(1,252):
    df_p_merge.iloc[[i],[2]] = np.array(df_p_merge.iloc[[i-1],[2]])/np.array(df_p_merge.iloc[[0],[4]]) * np.array(df_p_merge.iloc[[0],[5]])    # (前一天卖完)钱 / 最低价 * 最高价 = 等于当天的全部收入
    df_p_merge.iloc[[i],[3]] = np.array(df_p_merge.iloc[[i-1],[2]])/np.array(df_p_merge.iloc[[0],[4]]) * np.array(df_p_merge.iloc[[0],[8]])   # (前一天卖完)钱 / 最低价 * 差价 = 等于当天的净收益
df = df_p_merge.groupby('symbol')['net_income'].sum()

df = df.reset_index()
print('2016年最赚钱的股票及净收益如下：')
print(df[df['net_income'] == df['net_income'].max()])